# Training Set Creation (Final Model)

This notebook constructs the training dataset that will later be used to build and evaluate the final model.  

The **input file** is:  
- `dataset_dfs10000.pkl` (containing the raw sequences).  

The **output file** generated by this notebook is:  
- `trainingDataMax10000.csv`.  

---

## Metrics Required

In order to train the model, it is necessary to determine the **most comparison-efficient sorting algorithm** for each sequence, which serves as the target label. As input features, we employ the **sampled presortedness metrics** (**Runs** and **Deletions**) together with the **sequence length**.  

For the evaluation of the model, we additionally record the **number of comparisons required** to compute the presortedness metrics, as this reflects the computational overhead associated with feature extraction.  

---

## Sampling Strategies

To address **RQ3**, the notebook implements different sampling strategies and sample sizes. Specifically, the following methods can be applied:  

- `sampling_10_step(sample_size)`  
- `sampling_30_step(sample_size)`  

These procedures allow us to investigate how different sampling strategies and sample sizes influence the model.

---


## Sorting algorithm (comparison benchmark)

### Insertion sort

In [1]:
def insertion_sort(A):
    comparisons = 0
    for i in range(1, len(A)):
        key = A[i]
        j = i - 1
        comparisons += 1
        while j >= 0 and A[j] > key:
            comparisons += 1
            A[j + 1] = A[j]
            j -= 1
        A[j + 1] = key
    return comparisons

### Merge sort

In [3]:
def merge_sort(A):
    if len(A) <= 1:
        return 0
    mid = len(A) // 2
    left = A[:mid]
    right = A[mid:]

    comparecount = merge_sort(left) + merge_sort(right)

    i = 0
    j = 0
    k = 0
    
    while i < len(left) and j < len(right):
        comparecount += 1
        if left[i] <= right[j]:
            A[k] = left[i]
            i += 1
        else:
            A[k] = right[j]
            j += 1
        k += 1

    while i < len(left):
        comparecount += 1
        A[k] = left[i]
        i += 1
        k += 1

    while j < len(right):
        comparecount += 1
        A[k]=right[j]
        j += 1
        k += 1

    return comparecount

### Timsort

In [6]:
MINIMUM = 32

def find_minrun(n): 
    r = 0
    while n >= MINIMUM: 
        r |= n & 1
        n >>= 1
    return n + r 

def tim_insertion_sort(array, left, right): 
    global comparisons
    for i in range(left + 1, right + 1):
        key = array[i]
        j = i - 1
        comparisons += 1
        while j >= left and key < array[j]:
            array[j + 1] = array[j]
            j -= 1
            comparisons += 1
        array[j + 1] = key
    return array
              
def tim_merge(array, l, m, r): 
    global comparisons
    array_length1 = m - l + 1
    array_length2 = r - m 
    left = []
    right = []
    for i in range(array_length1): 
        left.append(array[l + i]) 
    for i in range(array_length2): 
        right.append(array[m + 1 + i]) 
  
    i = 0
    j = 0
    k = l
   
    while j < array_length2 and i < array_length1: 
        if left[i] <= right[j]: 
            array[k] = left[i] 
            i += 1
        else: 
            array[k] = right[j] 
            j += 1
        k += 1
        comparisons += 1
  
    while i < array_length1: 
        array[k] = left[i] 
        k += 1
        i += 1
        comparisons += 1
  
    while j < array_length2: 
        array[k] = right[j] 
        k += 1
        j += 1
        comparisons += 1
  
def timsort(array): 
    n = len(array) 
    minrun = find_minrun(n) 
  
    for start in range(0, n, minrun): 
        end = min(start + minrun - 1, n - 1) 
        tim_insertion_sort(array, start, end) 
   
    size = minrun 
    while size < n: 
        for left in range(0, n, 2 * size): 
            mid = min(n - 1, left + size - 1) 
            right = min((left + 2 * size - 1), (n - 1)) 
            tim_merge(array, left, mid, right) 
        size = 2 * size

    return comparisons

## PRESORTEDNESS Metrics

### Number of Runs
The number of runs, is the number of increasing sequences in an array minus one

In [8]:
def runs(arr):
    count = 0

    for key in range(1,len(arr)):
        if arr[key] < arr[key-1]:
            count += 1

    return count

0


Number of comparisons needed for Runs computation

In [9]:
def runs_comp(arr):
    count = 0
    comparisons = 0
    for key in range(1,len(arr)):
        comparisons += 1
        if arr[key] < arr[key-1]:
            count += 1

    return comparisons

### Number of Deletions
The minimum number of elements that need to be removed from array to obtain a sorted sequence

In [10]:
def deletions(arr):
    def ceil_index(sub, val):
        l, r = 0, len(sub)-1
        while l <= r:
            mid = (l + r) // 2
            if sub[mid] >= val:
                r = mid - 1
            else:
                l = mid + 1
        return l
 
    sub = [arr[0]]
    for i in range(1, len(arr)):
        if arr[i] >= sub[-1]:
            sub.append(arr[i])
        else:
            sub[ceil_index(sub, arr[i])] = arr[i]
 
    return len(arr) - len(sub)

0


Number of comparisons needed for Deletions computation

In [11]:
def deletions_comp(arr):
    global comparisons
    comparisons = 0
    def ceil_index(sub, val):
        global comparisons
        l, r = 0, len(sub)-1
        while l <= r:
            mid = (l + r) // 2
            comparisons += 1
            if sub[mid] >= val:
                r = mid - 1
            else:
                l = mid + 1
        return l
 
    sub = [arr[0]]
    for i in range(1, len(arr)):
        comparisons += 1
        if arr[i] >= sub[-1]:
            sub.append(arr[i])
        else:
            sub[ceil_index(sub, arr[i])] = arr[i]
 
    return comparisons

## Training Set creation (sorting)

In [18]:
import pickle
import os
import pandas as pd
import numpy as np
import math
import random
import matplotlib.pyplot as plt
random.seed(42)
np.random.seed(42)


results = []

with open('dataset_dfs10000.pkl', 'rb') as f:
    dataset_dfs = pickle.load(f)

for key, df in dataset_dfs.items():
    for column in df.columns:
        arr = df[column].values
        if len(arr) < 400:
            continue
        
        # sorting algorithm comparison calculation
        comp_merge = merge_sort(arr.copy())
        comp_insertion = insertion_sort(arr.copy())
        global comparisons
        comparisons = 0
        comp_tim = timsort(arr.copy())
                
        comparison_counts = {
            'insertion_sort': comp_insertion,
            'merge_sort': comp_merge,
            'timsort': comp_tim,
        }

        min_algorithm = min(comparison_counts, key=comparison_counts.get)
        min_comparisons = comparison_counts[min_algorithm]

        # RQ3 sampling_10_step and sampling_30_step have been used to evaluate optimal sampling strategy and sample size

        def sampling_10_step(sample_size):
            return [arr[i] for i in range(0, sample_size * 10, 10)]

        def sampling_30_step(sample_size):
            step = 30

            sample = arr[: (sample_size - 10) * step : step]

            reverse_sampled = arr[-10 * step::step]
    
            result = np.concatenate((sample, reverse_sampled))
            
            return result

        # RQ4 sampling for final model

        def samplingDistStatic():
            first_20 = arr[:200:10]

            middle_start = len(arr) // 2  
            middle_10 = arr[middle_start:middle_start + 100 :10]

            last_10 = arr[-100::10]

            return np.concatenate([first_20, middle_10, last_10])

        results.append({
            'Dataset': key,
            'Column': column,
            'Algorithm': min_algorithm,
            'Comparisons': min_comparisons,
            
            'deletions_val_distStatic': deletions(samplingDistStatic()),
            'runs_val_distStatic': runs(samplingDistStatic()),
            'deletions_comp_distStatic': deletions_comp(samplingDistStatic()),
            'runs_comp_distStatic': runs_comp(samplingDistStatic()),
            
            'arr_len': len(arr),
            
            'insertion_sort': comp_insertion,
            'merge_sort': comp_merge,
            'timsort': comp_tim,
        })


df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv('trainingDataMax10000.csv')

         Dataset      Column       Algorithm  Comparisons  \
0     cp_ratings  Unnamed: 0  insertion_sort         1251   
1     cp_ratings  max_rating      merge_sort        12976   
2     cp_ratings    contest1      merge_sort        12976   
3     cp_ratings    contest2      merge_sort        12976   
4     cp_ratings    contest3      merge_sort        12976   
...          ...         ...             ...          ...   
1685       train      Pclass         timsort         6662   
1686       train         Age      merge_sort         6830   
1687       train       SibSp         timsort         6242   
1688       train       Parch         timsort         5904   
1689       train        Fare      merge_sort         6830   

      deletions_val_distStatic  runs_val_distStatic  \
0                            0                    0   
1                           31                   18   
2                           31                   17   
3                           33                 